<a href="https://colab.research.google.com/github/usamasafdarvirk/A_Deep_Learning_Approach_to_Precise_MRI_Brain_Tumor_Diagnosis/blob/main/MRI_Image_Classification_Using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the Required Libraries

In [ ]:
import os               # For Acessing Image Folders
from PIL import Image   # Fot Loading Images from Folders

# Importing Google Drive

In [ ]:
def load_images_with_labels(image_folder):
    """
    Loads images and their labels from a specified folder, where each subfolder's name
    is used as the label for all images within that subfolder.

    Parameters:
    - image_folder: str. A path to the folder containing subfolders of images.

    Returns:
    - images: list of PIL.Image objects.
    - labels: list of str, each corresponding to the label of the images.
    """
    images = []
    labels = []
    # Check for valid image extensions
    valid_extensions = ('.jpg')

    # Loop through all the directories in the image directory
    for folder_name in os.listdir(image_folder):
        folder_path = os.path.join(image_folder, folder_name)
        if os.path.isdir(folder_path):  # Ensure it is a directory
            for filename in os.listdir(folder_path):
                if filename.endswith(valid_extensions):
                    # Full path to the image file
                    image_path = os.path.join(folder_path, filename)

                    # Load the image from the filesystem
                    image = Image.open(image_path)
                    image = image.convert('RGB')  # Ensure image is in RGB format

                    # The label is the name of the folder
                    label = folder_name

                    # Append the image and label to the lists
                    images.append(image)
                    labels.append(label)

    return images, labels

# Example usage:
# Specify the path to your image directory
training_folder = '/content/drive/MyDrive/MRI_Brain_Image_Classification/'
testing_folder = '/content/drive/MyDrive/MRI_Brain_Image_Classification/'
training_images, training_labels = load_images_with_labels(training_folder)
testing_images, testing_labels = load_images_with_labels(testing_folder)
print("Loaded Training ", len(training_images), "images.")
print("Training Labels:", training_labels)

Loaded Training  0 images.
Training Labels: []
